<h2>Spark-SQL</h2>

cpu logs

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions

In [3]:
spark=SparkSession.builder.appName("spark sql").config("spark.driver.extraClassPath", "/opt/spark/jars/mysql-connector-j-8.0.33.jar").getOrCreate()

23/07/21 14:58:25 WARN Utils: Your hostname, ShreyashUbuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/21 14:58:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/21 14:58:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df=spark.read.csv('hdfs://localhost:9000/pysql/*.csv',header=True,inferSchema=True)


In [5]:
df.createOrReplaceTempView("df")

23/07/21 14:59:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Find Idle Computers From Data

In [6]:
max_idle=spark.sql("select user_name,MAX(idle_time) AS max_idle_time FROM (SELECT user_name,DateTime,UNIX_TIMESTAMP(DateTime)-UNIX_TIMESTAMP(LAG(DateTime) OVER (PARTITION BY user_name ORDER BY DateTime)) AS idle_time FROM df WHERE keyboard='0.0' and mouse='0.0') subquery GROUP BY user_name ORDER BY max_idle_time DESC")
max_idle.show()

+--------------------+-------------+
|           user_name|max_idle_time|
+--------------------+-------------+
|deepshukla292@gma...|       141300|
|markfernandes66@g...|        85500|
|sharlawar77@gmail...|        84300|
|rahilstar11@gmail...|        83400|
|bhagyashrichalke2...|        83400|
|salinabodale73@gm...|        82500|
|damodharn21@gmail...|        80100|
|  iamnzm@outlook.com|        75600|
+--------------------+-------------+



In [7]:
# Configure the connection properties 
from py4j.protocol import Py4JJavaError
database_url = "jdbc:mysql://localhost:3306/pyspark"
table_name = "idle_user"
properties = {
    "user": "root",
    "password": "Admin@123",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Write the DataFrame to the existing MySQL database table (append mode)
max_idle.write \
    .format("jdbc") \
    .option("url", database_url) \
    .option("dbtable", table_name) \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .mode("append") \
    .save()


Find Users with lowest number of average hours

In [8]:
lowest_avg_hours=spark.sql("""  SELECT user_name,AVG(hour(DateTime)) AS avg_hours
                                    FROM df 
                                    GROUP BY user_name 
                                    ORDER BY avg_hours """)
lowest_avg_hours.show()

+--------------------+------------------+
|           user_name|         avg_hours|
+--------------------+------------------+
|  iamnzm@outlook.com|13.785016286644952|
|markfernandes66@g...|14.100393700787402|
|deepshukla292@gma...|14.130973451327433|
|sharlawar77@gmail...|14.205172413793104|
|damodharn21@gmail...|14.312252964426877|
|bhagyashrichalke2...|14.352697095435685|
|rahilstar11@gmail...| 14.37568058076225|
|salinabodale73@gm...|14.576449912126538|
+--------------------+------------------+



In [9]:
# Configure the connection properties 
from py4j.protocol import Py4JJavaError
database_url = "jdbc:mysql://localhost:3306/pyspark"
table_name = "lowest_avg"
properties = {
    "user": "root",
    "password": "Admin@123",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Write the DataFrame to the existing MySQL database table (append mode)
lowest_avg_hours.write \
    .format("jdbc") \
    .option("url", database_url) \
    .option("dbtable", table_name) \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .mode("append") \
    .save()


Find Users with highest number of average hours

In [6]:
highest_avg_hours=spark.sql("""SELECT user_name,AVG(hour(DateTime)) AS avg_hours
                                    FROM df
                                    GROUP BY user_name 
                                    ORDER BY avg_hours DESC""")
highest_avg_hours.show()

+--------------------+------------------+
|           user_name|         avg_hours|
+--------------------+------------------+
|salinabodale73@gm...|14.576449912126538|
|rahilstar11@gmail...| 14.37568058076225|
|bhagyashrichalke2...|14.352697095435685|
|damodharn21@gmail...|14.312252964426877|
|sharlawar77@gmail...|14.205172413793104|
|deepshukla292@gma...|14.130973451327433|
|markfernandes66@g...|14.100393700787402|
|  iamnzm@outlook.com|13.785016286644952|
+--------------------+------------------+



In [7]:
# Configure the connection properties 
from py4j.protocol import Py4JJavaError
database_url = "jdbc:mysql://localhost:3306/pyspark"
table_name = "highest_avg"
properties = {
    "user": "root",
    "password": "Admin@123",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Write the DataFrame to the existing MySQL database table (append mode)
highest_avg_hours.write \
    .format("jdbc") \
    .option("url", database_url) \
    .option("dbtable", table_name) \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .mode("append") \
    .save()


Find Users with late comings

In [11]:
spark.sql("""select user_name,DateTime
           FROM df 
           WHERE datetime>'2019-09-16 13:00:00'
           """).show()


+--------------------+-------------------+
|           user_name|           DateTime|
+--------------------+-------------------+
|  iamnzm@outlook.com|2019-09-19 08:40:02|
|  iamnzm@outlook.com|2019-09-19 08:45:02|
|  iamnzm@outlook.com|2019-09-19 08:50:01|
|  iamnzm@outlook.com|2019-09-19 08:55:01|
|  iamnzm@outlook.com|2019-09-19 09:00:01|
|  iamnzm@outlook.com|2019-09-19 09:05:01|
|deepshukla292@gma...|2019-09-19 09:05:01|
|  iamnzm@outlook.com|2019-09-19 09:10:01|
|deepshukla292@gma...|2019-09-19 09:10:01|
|  iamnzm@outlook.com|2019-09-19 09:15:02|
|deepshukla292@gma...|2019-09-19 09:15:02|
|markfernandes66@g...|2019-09-19 09:15:01|
|markfernandes66@g...|2019-09-19 09:10:01|
|markfernandes66@g...|2019-09-19 09:20:01|
|  iamnzm@outlook.com|2019-09-19 09:20:02|
|deepshukla292@gma...|2019-09-19 09:20:02|
|markfernandes66@g...|2019-09-19 09:25:01|
|  iamnzm@outlook.com|2019-09-19 09:25:01|
|deepshukla292@gma...|2019-09-19 09:25:01|
|markfernandes66@g...|2019-09-19 09:30:01|
+----------

In [9]:
late_users_df = spark.sql("""
    SELECT user_name, COUNT(*) AS late_count,
           RANK() OVER (ORDER BY COUNT(*) DESC) AS user_rank
    FROM df
    WHERE hour(DateTime) > 9 OR (hour(DateTime) = 9 AND minute(DateTime) > 30)
    GROUP BY user_name
    ORDER BY late_count DESC
""")

# Display the result
late_users_df.show()

23/07/21 15:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+---------+
|           user_name|late_count|user_rank|
+--------------------+----------+---------+
|  iamnzm@outlook.com|       576|        1|
|salinabodale73@gm...|       569|        2|
|sharlawar77@gmail...|       568|        3|
|rahilstar11@gmail...|       551|        4|
|deepshukla292@gma...|       547|        5|
|markfernandes66@g...|       496|        6|
|bhagyashrichalke2...|       482|        7|
|damodharn21@gmail...|       253|        8|
+--------------------+----------+---------+



In [10]:
# Configure the connection properties 
from py4j.protocol import Py4JJavaError
database_url = "jdbc:mysql://localhost:3306/pyspark"
table_name = "late_commers"
properties = {
    "user": "root",
    "password": "Admin@123",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Write the DataFrame to the existing MySQL database table (append mode)
late_users_df.write \
    .format("jdbc") \
    .option("url", database_url) \
    .option("dbtable", table_name) \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .mode("append") \
    .save()


23/07/21 15:02:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [11]:
spark.stop()